<a href="https://colab.research.google.com/github/PreshitaDave/QSAR-model-for-FLT1/blob/main/QSAR_Flt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PART** **1 - DATA COLLECTION**

This is to Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

Import the necessary libraries

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

Target Search

In [ ]:
target = new_client.target
target_query = target.search('Flt1')
targets = pd.DataFrame.from_dict(target_query)

Select the required target

In [ ]:
selected_target = targets.target_chembl_id[0]

Here, we will retrieve only bioactivity data for FLT1(CHEMBL1868) that are reported as IC50  values in nM (nanomolar) unit.

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
df = pd.DataFrame.from_dict(res)

In [ ]:
df.to_csv('bioactivity_data.csv', index=False)

Firstly, we need to mount the Google Drive into Colab so that we can have access to our Google adrive from within Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Next, we create a data folder in our Colab Notebooks folder on Google Drive.

In [ ]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/Major_project"

In [ ]:
! cp bioactivity_data.csv "/content/gdrive/My Drive/Colab Notebooks/Major_project"
! ls -l "/content/gdrive/My Drive/Colab Notebooks/Major_project"
! ls
! head bioactivity_data.csv

**DATA PRE-PROCESSING OF THE BIOACTIVITY DATA**

Handling Missing data

In [ ]:
df2 = df[df.standard_value.notna()]
df2_final = df2[df2.canonical_smiles.notna()]
df2_final = df2_final.reset_index()

Labeling compounds as either being active, inactive or intermediate

Labeling compounds as either being active or inactive.
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be active while those greater than 1000 nM will be considered to be inactive.

In [ ]:
bioactivity_class = []
for i in df2_final.standard_value:
  if float(i) > 1000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")

Iterate the molecule_chembl_id to a list

In [ ]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df3 = df2_final[selection]
df3['bioactivity_class'] = pd.Series(bioactivity_class)

In [ ]:
df3.to_csv('bioactivity_preprocessed_data.csv', index=False)

In [ ]:
! cp bioactivity_preprocessed_data.csv "/content/gdrive/My Drive/Colab Notebooks/Major_project"

In [ ]:
! ls "/content/gdrive/My Drive/Colab Notebooks/Major_project"

# **PART** **2 - EXPLORATORY DATA ANALYSIS**

Install conda and rdkit libraries

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

Load the bioactivity data

In [ ]:
import pandas as pd
df = pd.read_csv('bioactivity_preprocessed_data.csv')
df=df.dropna()
df.shape

Import other libraries

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

Calculate Lipinski descriptors

In [ ]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

df_lipinski = lipinski(df.canonical_smiles)
df_lipinski


In [ ]:
#combine both dfs
df_combined = pd.concat([df,df_lipinski], axis=1)

Convert IC50 to pIC50

In [ ]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

df_combined

In [ ]:
df_combined.standard_value.describe()

Point to note: Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.

In [ ]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

We will first apply the norm_value() function so that the values in the standard_value column is normalized.

In [ ]:
df_norm = norm_value(df_combined)

In [ ]:
df_norm.standard_value_norm.describe()

In [ ]:
df_final = pIC50(df_norm)

In [ ]:
df_final.to_csv('bioactivity_data_3class_pIC50.csv')

In [ ]:
df_final.pIC50.describe()

Importing libraries for visualization

In [ ]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

Frequency plot of bioactive classes

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.countplot(x='bioactivity_class', data=df_2class, edgecolor='black')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('plot_bioactivity_class.pdf')

Scatter plot of MW versus LogP

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.scatterplot(x='MW', y='LogP', data=df_2class, hue='bioactivity_class', size='pIC50', edgecolor='black', alpha=0.7)

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')

Creating Box Plots for the following: 

(a) pIC50 value

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'pIC50', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_ic50.pdf')

(b) MW

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'MW', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.pdf')

(c) LogP

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'LogP', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.pdf')

(d) NumHDonors

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHDonors', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHDonors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDonors.pdf')

(e) NumHAcceptors

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHAcceptors', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHAcceptors', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHAcceptors.pdf')

Downloading all files

In [ ]:
! zip -r results.zip . -i *.csv *.pdf

# **PART 3 - DESCRIPTOR DATA PREPARATION**



Download PaDEL-Descriptor

In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

In [ ]:
! unzip padel.zip

Load bioactivity data

In [ ]:
import pandas as pd
df3 = pd.read_csv('bioactivity_data_3class_pIC50.csv')

In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
df3_selection = df3[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

Calculate PaDEL descriptors

In [ ]:
! cat padel.sh

In [ ]:
! bash padel.sh

**Preparing the X and Y data matrices**

**Preparing X data matrix**

In [ ]:
df3_X = pd.read_csv('descriptors_output.csv')

In [ ]:
df3_X = df3_X.drop(columns=['Name'])

**Y variable**

Convert IC50 to pIC50

In [ ]:
df3_Y = df3['pIC50']

*Combining X and Y variable*

In [ ]:
dataset3 = pd.concat([df3_X,df3_Y], axis=1)

Save the csv file

In [ ]:
dataset3.to_csv('bioactivity_data_3class_pIC50_pubchem_fp.csv', index=False)

In [ ]:
! cp bioactivity_data_3class_pIC50_pubchem_fp.csv "/content/gdrive/My Drive/Colab Notebooks/Major_project"

**Generating the descriptor dataset for Ledum Palustre metabolites**

Remove the molecule.smi (contains the training dataset SMILES) before calculating descriptors for smiles.smi (contains the smiles for the 36 metabolites from Ledum Palustre)

In [ ]:
!rm molecule.smi

In [ ]:
import pandas as pd
ledum_pal=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Major_project/smiles_wo_dupli.smi", header=None)
ledum_pal.to_csv('smiles_ledum.smi', sep='\t', index=False, header=False)

In [ ]:
!bash padel.sh

In [ ]:
smiles_X = pd.read_csv('descriptors_output.csv')
smiles_X.to_csv('descriptors_output_ledum.csv', index = False)
! cp descriptors_output_ledum.csv "/content/gdrive/My Drive/Colab Notebooks/Major_project"

# **PART 4 - REGRESSION MODELS WITH RANDOM FOREST** 

Import the required libraries

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df_4 = pd.read_csv('bioactivity_data_3class_pIC50_pubchem_fp.csv')

Input Features

In [ ]:
X = df_4.drop('pIC50', axis=1)

Output features

In [ ]:
Y = df_4.pIC50

In [ ]:
smiles_X_ledum = pd.read_csv('descriptors_output_ledum.csv')
smiles_X_ledum = smiles_X_ledum.drop('Name', axis=1)

Removing low variance features

In [ ]:
from sklearn.feature_selection import VarianceThreshold
selection_vt = VarianceThreshold(threshold=(.8 * (1 - .8)))   
_ = selection_vt.fit(X)
mask = selection_vt.get_support()
X = X.loc[:, mask]

In [ ]:
X_columns = X.columns.tolist()

In [ ]:
smiles_X_ledum = smiles_X_ledum.filter(X_columns)

Data split (80/20)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
X_test.shape, Y_test.shape

Building a Regression Model using Random Forest

In [ ]:
import numpy as np
np.random.seed(100)
model = RandomForestRegressor(n_estimators=250, oob_score=True, max_depth=10)
model.fit(X_train, Y_train)
r2 = model.score(X_test, Y_test)

In [ ]:
print(f'Out-of-bag score estimate: {model.oob_score_:.3}')

In [ ]:
Y_pred = model.predict(X_test)


**Determine Performance Metrics**

Predicting Mean Absolute Error (MAE)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(Y_test, Y_pred)

Predicting Mean Squared Error (MSE)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, Y_pred)

Predicting Root Mean Squared Error (RMSE)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, Y_pred, squared = False)

Scatter Plot of Experimental vs Predicted pIC50 Values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(color_codes=True)
sns.set_style("white")

ax = sns.regplot(Y_test, Y_pred, scatter_kws={'alpha':0.4})
ax.set_xlabel('Experimental pIC50', fontsize='large', fontweight='bold')
ax.set_ylabel('Predicted pIC50', fontsize='large', fontweight='bold')
ax.set_xlim(0, 12)
ax.set_ylim(0, 12)
ax.figure.set_size_inches(5, 5)
plt.show

**Creating the Feature Importance graph**

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

figure(num=None, figsize=(7,7), dpi=80, facecolor='w', edgecolor='k')

feat_importances = pd.Series(model.feature_importances_, index= X.columns)
feat_importances.nlargest(10).plot(kind='barh')


plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.show()

Creating the list of the top 10 important features

In [ ]:
top_10=feat_importances.nlargest(10)
top_10_list = top_10.index.to_list()

**Predicting the pIC50 values for the Ledum Palustre metabolites**

In [ ]:
Y_pred_smiles = model.predict(smiles_X_ledum)

In [ ]:
import numpy as np
ledum_pal.columns = ['Smiles']

Y_pred_smiles = pd.DataFrame(Y_pred_smiles)
Y_pred_smiles.columns = ['pIC50']

df_ledum = pd.concat([ledum_pal, Y_pred_smiles], axis = 1)

df_ledum.to_csv('Final_ledum_pal.csv', index = False)
! cp Final_ledum_pal.csv "/content/gdrive/My Drive/Colab Notebooks/Major_project"